# Create Meeting Minutes from an Audio File

This notebook transcribes meeting audio using the **Hugging Face pipeline** (Whisper) and generates structured meeting minutes using Llama 3.2 3B.

**Requirements:** Run in Google Colab with T4 GPU. Add `HF_TOKEN` to Colab secrets. Place audio file at `MyDrive/llms/denver_extract.mp3` on Google Drive.

## Google Colab Link: 

https://colab.research.google.com/drive/1CxXxkUXocVqVu1TihyFHjW1KZK-q5OHF?usp=sharing

## Pro-tip for Colab

If you see: *"Runtime error: CUDA is required but not available for bitsandbytes"* — this usually means Colab switched your runtime. Fix:

1. Kernel → Disconnect and delete runtime
2. Edit → Clear All Outputs, reconnect
3. Select T4 GPU (View resources to confirm)
4. Rerun from the top

In [ ]:
!pip install -q --upgrade bitsandbytes accelerate transformers==4.57.6

In [ ]:
# Imports
from IPython.display import Markdown, display
from google.colab import drive, userdata
from huggingface_hub import login
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TextStreamer,
    BitsAndBytesConfig,
    pipeline
)
import torch

In [ ]:
# Constants
LLAMA = "meta-llama/Llama-3.1-8B-Instruct"
WHISPER_MODEL = "openai/whisper-medium.en"

In [ ]:
# Mount Google Drive (same path as reference)
# Place denver_extract.mp3 at: MyDrive/llms/denver_extract.mp3
drive.mount("/content/drive")
audio_filename = "/content/drive/MyDrive/llms/denver_extract.mp3"

In [ ]:
# Sign in to HuggingFace Hub (HF_TOKEN stored in Colab secrets)
hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

## Step 1: Transcribe Audio (Hugging Face Pipeline)

Uses Whisper via the transformers pipeline for transcription.

In [ ]:
pipe = pipeline(
    "automatic-speech-recognition",
    model=WHISPER_MODEL,
    torch_dtype=torch.float16,
    device=0 if torch.cuda.is_available() else "cpu",
    return_timestamps=True
)

result = pipe(audio_filename)
transcription = result["text"]
print("Transcription:")
print(transcription)

## Step 2: Generate Meeting Minutes (Llama 3.2 3B)

In [ ]:
system_message = """
You produce minutes of meetings from transcripts, with summary, key discussion points,
takeaways and action items with owners, in markdown format without code blocks.
"""

user_prompt = f"""
Below is an extract transcript of a Denver council meeting.
Please write minutes in markdown without code blocks, including:
- a summary with attendees, location and date
- discussion points
- takeaways
- action items with owners

Transcription:
{transcription}
"""

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
]

In [ ]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    LLAMA,
    device_map="auto",
    quantization_config=quant_config
)

inputs = tokenizer.apply_chat_template(
    messages,
    return_tensors="pt",
    add_generation_prompt=True
).to("cuda")

streamer = TextStreamer(tokenizer)
outputs = model.generate(
    inputs,
    max_new_tokens=2000,
    streamer=streamer,
    do_sample=False
)

In [ ]:
# Extract assistant response (remove input prompt)
generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
# Get text after the last [INST] block
if "[/INST]" in generated:
    response = generated.split("[/INST]")[-1].strip()
else:
    response = generated

display(Markdown(response))